In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

from IPython.display import clear_output
from datetime import datetime

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./1.page.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_['Page'] = input_['Page'].astype(int)

print(f'总数量：{len(input_)}')
print()

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)

# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error

    while not work.empty():
        a = work.get_nowait()

        # = = = = = = = = = = = = = = =

        crawler_status = 'error'

        # = = = = = = = = = = = = = = =

        import requests
        import json

        import sys
        sys.path.append('../00.Tools')
        from crawler_configuration import get_header, get_proxy

        from bs4 import BeautifulSoup
        from lxml import etree

        import json

        try:
            b = 0
            while True:

                while True:
                    b += 1
                    try:
                        resp = requests.get(input_.loc[a, 'Page Url'],
                                            headers=get_header(),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))
                        break
                    except:
                        continue

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    # = = = = = = = = = = = = = = =

                    list_data = json.loads(html.xpath('//div[@data-a-card-type="basic"]/div[@class="p13n-desktop-grid"]/@data-client-recs-list')[0])

                    if list_data:
                        break

            # = = = = = = = = = = = = = = =

            df_temp = pd.DataFrame({'Market': input_.loc[a, 'Market'],
                                    'Page': input_.loc[a, 'Page'],
                                    'Page Url': input_.loc[a, 'Page Url'],
                                    'No': [i+1 for i in range(len(list_data))],
                                    'ASIN': [data['id'] for data in list_data]})

            output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            crawler_status = 'ok'
        except:
            df_temp = pd.DataFrame([{'Market': input_.loc[a, 'Market'],
                                     'Page': input_.loc[a, 'Page'],
                                     'Page Url': input_.loc[a, 'Page Url']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        # = = = = = = = = = = = = = = =

        if work.qsize() % 1_000 == 0 and work.qsize() != 0:
            clear_output()

        print(f'''[状态：{crawler_status}，尝试次数：{b:}] - {input_.loc[a, 'Page']}.{input_.loc[a, 'Page Url']}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(12):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
if not output_correct.empty:
    output_correct = output_correct.sort_values(by=['Page', 'No'],
                                                ascending=[True, True],
                                                ignore_index=True)
    output_correct = output_correct.drop_duplicates(['ASIN'],
                                                    keep='first',
                                                    ignore_index=True)
    output_correct['No'] = [i+1 for i in range(len(output_correct))]
    output_correct.to_excel(f'''./2.menu-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
if not output_error.empty:
    output_error = output_error.sort_values(by=['Page'],
                                            ascending=[True],
                                            ignore_index=True)
    output_error.to_excel(f'./menu_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')

总数量：2

[状态：ok，尝试次数：2] - 2.https://www.amazon.com/gp/bestsellers/automotive/15730511/ref=pd_zg_hrsr_automotive?language=en_US&pg=2
[剩余数量：0] - [当前时间：19:13:25]

[状态：ok，尝试次数：10] - 1.https://www.amazon.com/gp/bestsellers/automotive/15730511/ref=pd_zg_hrsr_automotive?language=en_US&pg=1
[剩余数量：0] - [当前时间：19:13:25]

输出ing...

Done ~
